In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=a2905785-2a82-4cb1-be4a-48d876c2913e
To: /kaggle/working/data.pkl
100%|██████████████████████████████████████| 1.24G/1.24G [00:50<00:00, 24.7MB/s]


In [3]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import label_creator
from preprocess import data_filtering
from preprocess import resampling
from preprocess import including_data
from models import ribeiro2
from models import CompModel

In [4]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [5]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 7714.07it/s]


In [6]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)

100%|██████████| 10344/10344 [00:00<00:00, 880109.55it/s]
10344it [00:07, 1327.51it/s]
10344it [00:04, 2247.26it/s]


In [7]:
x_train = np.reshape(x_train, (5000, 100,12))
x_test = np.reshape(x_test, (1126, 100,12))

In [8]:
model = ribeiro2()
model, lr_sch = CompModel(model)

history = model.fit(x_train, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100
157/157 [==============================] - 19s 25ms/step - loss: 0.4454 - acc: 0.3088 - lr: 0.0010
Epoch 2/100
157/157 [==============================] - 3s 21ms/step - loss: 0.3927 - acc: 0.3890 - lr: 9.7724e-04
Epoch 3/100
157/157 [==============================] - 3s 21ms/step - loss: 0.3199 - acc: 0.5166 - lr: 9.5499e-04
Epoch 4/100
157/157 [==============================] - 3s 21ms/step - loss: 0.2619 - acc: 0.6076 - lr: 9.3325e-04
Epoch 5/100
157/157 [==============================] - 3s 22ms/step - loss: 0.2020 - acc: 0.7016 - lr: 9.1201e-04
Epoch 6/100
157/157 [==============================] - 4s 23ms/step - loss: 0.1515 - acc: 0.7722 - lr: 8.9125e-04
Epoch 7/100
157/157 [==============================] - 3s 22ms/step - loss: 0.1029 - acc: 0.8360 - lr: 8.7096e-04
Epoch 8/100
157/157 [==============================] - 3s 21ms/step - loss: 0.0656 - acc: 0.8814 - lr: 8.5114e-04
Epoch 9/100
157/157 [==============================] - 3s 22ms/step - loss: 0.0425 - acc: 0

In [9]:
model.save('/kaggle/working/ribeiro.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
